In [11]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (we will only train the top layers)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification (surgery or no surgery)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Prepare the data using ImageDataGenerator (Assuming the dataset is organized in subfolders for each class)
train_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/Users/College doc/python/archive (6)/train',  # Folder containing 'surgery' and 'no_surgery' subfolders
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/Users/College doc/python/archive (6)/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)

# Save the model for future use
model.save('bone_surgery_prediction_model.h5')

from tensorflow.keras.models import load_model

model = load_model('bone_surgery_prediction_model.h5')

from tensorflow.keras.preprocessing import image
import numpy as np

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values to [0, 1]
    return img_array

img_path = '/Users/yuvraj/Downloads/Unknown.jpeg'  # Replace with your image path
img_array = prepare_image(img_path)
prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("Predicted: Surgery")
else:
    print("Predicted: No Surgery")

Found 7091 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 22857s 95s/step - accuracy: 0.5014 - loss: 0.6985 - val_accuracy: 0.6250 - val_loss: 0.6419
Epoch 2/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 872s 4s/step - accuracy: 0.5000 - loss: 0.6813 - val_accuracy: 0.5000 - val_loss: 0.6609
Epoch 3/10
118/221 ━━━━━━━━━━━━━━━━━━━━ 2:05:13 73s/step - accuracy: 0.5303 - loss: 0.6864